# Fine-tuning Sentence Transformers на Cosine Similarity

Пайплайн для:
- Дообучение sentence-transformers моделей
- Triplet loss, Contrastive loss, CosineSimilarityLoss
- Evaluation на косинусном расстоянии
- Поиск похожих текстов

In [ ]:
!pip install sentence-transformers torch pandas numpy scikit-learn datasets -q

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from sentence_transformers import models, datasets as st_datasets
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат 1: пары текстов с оценкой похожести (0-1 или 0-5)
# Колонки: 'text1', 'text2', 'similarity_score'
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Или формат 2: triplets (anchor, positive, negative)
# Колонки: 'anchor', 'positive', 'negative'

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nПример данных:")
print(train_df.head())

## 2. Загрузка базовой модели

In [ ]:
# Выбор базовой модели
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'  # быстрая
# MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'  # качественная
# MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'  # мультиязычная

model = SentenceTransformer(MODEL_NAME)
print(f"✓ Модель загружена: {MODEL_NAME}")
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")

## 3. Подготовка данных для обучения

### Вариант A: CosineSimilarityLoss (пары с оценками)

In [ ]:
# Если у вас формат: text1, text2, similarity_score
def prepare_cosine_loss_data(df):
    """
    Подготовка данных для CosineSimilarityLoss
    similarity_score должен быть в диапазоне [0, 1] или [-1, 1]
    """
    examples = []
    
    for idx, row in df.iterrows():
        text1 = str(row['text1'])
        text2 = str(row['text2'])
        score = float(row['similarity_score'])
        
        # Нормализация score если нужно (например, если 0-5, делим на 5)
        # score = score / 5.0
        
        example = InputExample(texts=[text1, text2], label=score)
        examples.append(example)
    
    return examples

# Создаем примеры
train_examples = prepare_cosine_loss_data(train_df)
print(f"✓ Создано {len(train_examples)} training examples")
print(f"\nПример: {train_examples[0].texts}")
print(f"Label: {train_examples[0].label}")

### Вариант B: TripletLoss (anchor, positive, negative)

In [ ]:
# Если у вас формат triplets: anchor, positive, negative
def prepare_triplet_loss_data(df):
    """
    Подготовка данных для TripletLoss
    """
    examples = []
    
    for idx, row in df.iterrows():
        anchor = str(row['anchor'])
        positive = str(row['positive'])
        negative = str(row['negative'])
        
        example = InputExample(texts=[anchor, positive, negative])
        examples.append(example)
    
    return examples

# Раскомментируйте если используете triplet формат
# train_examples = prepare_triplet_loss_data(train_df)
# print(f"✓ Создано {len(train_examples)} triplet examples")

### Генерация triplets из пар (если нужно)

In [ ]:
def generate_triplets_from_pairs(df, similarity_threshold=0.7):
    """
    Генерация triplets из пар с оценками
    Высокая схожесть -> positive, низкая -> negative
    """
    triplets = []
    
    # Группируем похожие и непохожие пары
    similar_pairs = df[df['similarity_score'] >= similarity_threshold]
    dissimilar_pairs = df[df['similarity_score'] < similarity_threshold]
    
    for idx, row in similar_pairs.iterrows():
        anchor = str(row['text1'])
        positive = str(row['text2'])
        
        # Берем случайный negative из непохожих пар
        if len(dissimilar_pairs) > 0:
            neg_row = dissimilar_pairs.sample(1).iloc[0]
            negative = str(neg_row['text2'])
            
            triplets.append(InputExample(texts=[anchor, positive, negative]))
    
    return triplets

# Раскомментируйте если хотите использовать
# train_examples = generate_triplets_from_pairs(train_df, similarity_threshold=0.7)
# print(f"✓ Сгенерировано {len(train_examples)} triplets")

## 4. DataLoader и Loss Function

In [ ]:
# DataLoader
BATCH_SIZE = 16
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)

# Выбор loss function
# Вариант 1: CosineSimilarityLoss для пар с оценками
train_loss = losses.CosineSimilarityLoss(model)

# Вариант 2: TripletLoss для triplets
# train_loss = losses.TripletLoss(model, distance_metric=losses.TripletDistanceMetric.COSINE)

# Вариант 3: MultipleNegativesRankingLoss (популярный для retrieval)
# train_loss = losses.MultipleNegativesRankingLoss(model)

# Вариант 4: ContrastiveLoss
# train_loss = losses.ContrastiveLoss(model)

print(f"✓ Loss function: {type(train_loss).__name__}")
print(f"Batches per epoch: {len(train_dataloader)}")

## 5. Evaluator (опционально)

In [ ]:
# Создание валидационного сета
train_examples_split, val_examples_split = train_test_split(
    train_examples, test_size=0.1, random_state=42
)

# Evaluator для cosine similarity
sentences1 = [ex.texts[0] for ex in val_examples_split]
sentences2 = [ex.texts[1] for ex in val_examples_split]
scores = [ex.label for ex in val_examples_split]

evaluator = evaluation.EmbeddingSimilarityEvaluator(
    sentences1, sentences2, scores,
    name='cosine_eval'
)

# Обновляем train_dataloader для split данных
train_dataloader = DataLoader(train_examples_split, shuffle=True, batch_size=BATCH_SIZE)

print(f"✓ Evaluator создан")
print(f"Train: {len(train_examples_split)}, Val: {len(val_examples_split)}")

## 6. Fine-tuning

In [ ]:
# Параметры обучения
NUM_EPOCHS = 3
WARMUP_STEPS = int(len(train_dataloader) * NUM_EPOCHS * 0.1)
OUTPUT_DIR = './finetuned_sentence_transformer'

print(f"Параметры обучения:")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Warmup steps: {WARMUP_STEPS}")
print(f"Total steps: {len(train_dataloader) * NUM_EPOCHS}")

# Обучение
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=NUM_EPOCHS,
    evaluation_steps=len(train_dataloader) // 2,  # Evaluation каждые полэпохи
    warmup_steps=WARMUP_STEPS,
    output_path=OUTPUT_DIR,
    save_best_model=True,
    show_progress_bar=True
)

print(f"\n✓ Fine-tuning завершен!")
print(f"Модель сохранена в: {OUTPUT_DIR}")

## 7. Загрузка дообученной модели

In [ ]:
# Загружаем лучшую модель
finetuned_model = SentenceTransformer(OUTPUT_DIR)
print("✓ Дообученная модель загружена!")

## 8. Генерация эмбеддингов и предсказаний

In [ ]:
# Генерация эмбеддингов для test данных
test_texts1 = test_df['text1'].astype(str).tolist()
test_texts2 = test_df['text2'].astype(str).tolist()

print("Генерация эмбеддингов...")
embeddings1 = finetuned_model.encode(test_texts1, convert_to_numpy=True, show_progress_bar=True)
embeddings2 = finetuned_model.encode(test_texts2, convert_to_numpy=True, show_progress_bar=True)

# Вычисление cosine similarity
cosine_scores = []
for emb1, emb2 in zip(embeddings1, embeddings2):
    cos_sim = cosine_similarity([emb1], [emb2])[0][0]
    cosine_scores.append(cos_sim)

cosine_scores = np.array(cosine_scores)

print(f"\n✓ Предсказания готовы!")
print(f"Min similarity: {cosine_scores.min():.4f}")
print(f"Max similarity: {cosine_scores.max():.4f}")
print(f"Mean similarity: {cosine_scores.mean():.4f}")

## 9. Сравнение с базовой моделью

In [ ]:
# Предсказания базовой модели
base_model = SentenceTransformer(MODEL_NAME)

base_embeddings1 = base_model.encode(test_texts1[:100], convert_to_numpy=True)  # Для примера 100
base_embeddings2 = base_model.encode(test_texts2[:100], convert_to_numpy=True)

base_cosine_scores = []
for emb1, emb2 in zip(base_embeddings1, base_embeddings2):
    cos_sim = cosine_similarity([emb1], [emb2])[0][0]
    base_cosine_scores.append(cos_sim)

# Сравнение
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(base_cosine_scores, bins=30, alpha=0.7, label='Base Model')
axes[0].set_title('Базовая модель')
axes[0].set_xlabel('Cosine Similarity')
axes[0].set_ylabel('Frequency')
axes[0].legend()

axes[1].hist(cosine_scores[:100], bins=30, alpha=0.7, label='Fine-tuned Model', color='orange')
axes[1].set_title('Дообученная модель')
axes[1].set_xlabel('Cosine Similarity')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nBase model mean: {np.mean(base_cosine_scores):.4f}")
print(f"Fine-tuned model mean: {cosine_scores[:100].mean():.4f}")

## 10. Semantic Search (бонус)

In [ ]:
def semantic_search(query, corpus, model, top_k=5):
    """
    Поиск наиболее похожих текстов из корпуса
    """
    # Эмбеддинги
    query_embedding = model.encode(query, convert_to_numpy=True)
    corpus_embeddings = model.encode(corpus, convert_to_numpy=True, show_progress_bar=False)
    
    # Косинусное сходство
    similarities = cosine_similarity([query_embedding], corpus_embeddings)[0]
    
    # Топ-k результатов
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            'text': corpus[idx],
            'similarity': similarities[idx]
        })
    
    return results

# Пример использования
query = "машинное обучение и нейронные сети"  # === ВАШ ЗАПРОС ===
corpus = test_texts1[:1000]  # Корпус для поиска

results = semantic_search(query, corpus, finetuned_model, top_k=5)

print(f"\nЗапрос: {query}\n")
print("Топ-5 похожих текстов:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Similarity: {result['similarity']:.4f}")
    print(f"   Text: {result['text'][:200]}...")

## 11. Submission

In [ ]:
submission = pd.DataFrame({
    'id': test_df.index,  # или test_df['id']
    'similarity': cosine_scores
})

submission.to_csv('sentence_transformers_submission.csv', index=False)
print("\n✓ Submission сохранен!")
print(submission.head())